In [1]:
import json
import nltk
import string

from nltk.corpus import stopwords
from collections import Counter

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def remove_stopwords(token_list: list) -> list:
    stop_words = set(stopwords.words('portuguese'))
    further_punctuation = [
        ",", ".", ")", "(", "%", "|",
        "[", "]", "=", "'", "}", ":",
        ";", "˜"
    ]
    stop_words = stop_words.union(list(string.punctuation))
    stop_words = stop_words.union(further_punctuation)

    return [w for w in token_list if not w.lower() in stop_words]


In [4]:
CORPUS_PATH = "./corpus.json"

In [5]:
with open(CORPUS_PATH, "r", encoding="utf-8") as f:
    corpus = json.load(f)

In [7]:
total_sentencas = 0
total_tokens = 0
total_tokens_unicos = 0
tamanho_corpus = len(corpus)

for template in corpus:
    print("Título: ", template.get("titulo"))

    num_sentencas = len(template.get("artigo_sentenciado", 0))
    filtered_tokens = remove_stopwords(
        template.get("artigo_tokenizado", [])
    )
    num_tokens = len(filtered_tokens)
    num_tokens_unicos = len(set(filtered_tokens))
    pos_counts = Counter(template.get("pos_tagger", []))

    print("- Tokenização:")
    print("   Número de sentenças: ", num_sentencas)
    print("   Número de tokens: ", num_tokens)
    print("   Número de tokens únicos: ", num_tokens_unicos)
    print("- Etiquetagem POS:")
    print("   Número de substantivos comuns: ", pos_counts.get("NOUN", 0))
    print("   Número de substantivos próprios: ", pos_counts.get("PROPN", 0))
    print("   Número total de substantivos: ", pos_counts.get("NOUN", 0) + pos_counts.get("PROPN", 0))
    print("   Número de verbos: ", pos_counts.get("VERB", 0))
    print("   Número de adposições: ", pos_counts.get("ADJ", 0))
    print()
    print("-" * 50)

    total_sentencas += num_sentencas
    total_tokens += num_tokens
    total_tokens_unicos += num_tokens_unicos


Título:  Multipurpose Quantum Network Simulators: A comparative study
- Tokenização:
   Número de sentenças:  153
   Número de tokens:  1647
   Número de tokens únicos:  820
- Etiquetagem POS:
   Número de substantivos comuns:  733
   Número de substantivos próprios:  216
   Número total de substantivos:  949
   Número de verbos:  265
   Número de adposições:  244

--------------------------------------------------
Título:  Análise de desempenho de linguagens de programação e bibliotecas quânticas
- Tokenização:
   Número de sentenças:  93
   Número de tokens:  1170
   Número de tokens únicos:  563
- Etiquetagem POS:
   Número de substantivos comuns:  542
   Número de substantivos próprios:  164
   Número total de substantivos:  706
   Número de verbos:  193
   Número de adposições:  164

--------------------------------------------------
Título:  Optimization of a Polarization-Encoding System for Practical Quantum Key Distribution
- Tokenização:
   Número de sentenças:  197
   Número 

In [8]:
print("Estatísticas Agregadas:")
print("   Número médio de sentencas: ", float(total_sentencas) / tamanho_corpus)
print("   Número médio de tokens: ", float(total_tokens) / tamanho_corpus)
print("   Número médio de tokens únicos: ", float(total_tokens_unicos) / tamanho_corpus)

Estatísticas Agregadas:
   Número médio de sentencas:  165.95
   Número médio de tokens:  1743.75
   Número médio de tokens únicos:  781.7
